# BAX-422 Project 1
-- Anakin Liu, Jingyu Tang, Ruijie Shan, Ruiyuan Yang

<br/>

## Smart Travel Planner

#### APIs used:
* **OpenWhether** - https://openweathermap.org/api
* **Amadeus** - https://developers.amadeus.com/self-service

#### Problems/Questions Solved:
- **What is the current weather at my destination?**  
  *(Use OpenWeather API to fetch real-time weather data for a given location.)*
- **What are the cheapest flights available from my city?**  
  *(Use Skyscanner API to retrieve flight prices and compare options.)*
- **What are the available hotel options in the desired destination based on customer preferences?**  
  *(Use Amadeus Hotel List API to retrieve hotels that match customer preferences.)*

<br/>

### Libraries

In [1]:
import pandas as pd
import requests as rq
from datetime import datetime
from booking_scraper import get_hotel_detail

<br/>

### API Key Extract Function

In [2]:
def ext_api(file_name: str)->dict[str, str]:
    api_dict = {}
    with open(file_name, 'r') as file:
        for line in file:
            key, value = line.strip().split('=', 1)
            api_dict[key.strip()] = value.strip()
    return api_dict

<br/>

### Functionalities Class

In [10]:
class SmartTravelPlanner:
    def __init__(self, weather_key:str, amadeus_key_pack:tuple[str, str]):
        # Extract API keys
        self.__weather_key = weather_key

        # Base URLs for the APIs
        self.weather_base_url = "https://api.openweathermap.org/data/2.5/weather"
        self.amadeus_auth_url = "https://test.api.amadeus.com/v1/security/oauth2/token"
        self.amadeus_flight_url = "https://test.api.amadeus.com/v2/shopping/flight-offers"
        self.amadeus_hotel_list_url = "https://test.api.amadeus.com/v1/reference-data/locations/hotels/by-city"
        self.amadeus_hotel_ratings_url = "https://test.api.amadeus.com/v2/e-reputation/hotel-sentiments"
        self.amadeus_hotel_offer_url = "https://test.api.amadeus.com/v3/shopping/hotel-offers"

        # Initialize Amadeus access token
        payload = {
            "grant_type":"client_credentials",
            "client_id":amadeus_key_pack[0],
            "client_secret":amadeus_key_pack[1]
        }
        headers = {"Content-Type": "application/x-www-form-urlencoded"}
        response = rq.post(self.amadeus_auth_url, data=payload, headers=headers)
        self.__amadeus_token = response.json().get("access_token")
    


    # Generic method for making API calls and handling errors.
    def make_api_request(self, url:str, params:dict[str, str], headers=None)->dict|None:
        try:
            response = rq.get(url, params=params, headers=headers)
            return response.json()
        except rq.exceptions.RequestException as e:
            print(f"API request failed: {e}")
            return None
        
    

    # Method for timestamp formation
    def format_timestamp(self, timestamp:int, is_24h:bool)->str:
        dt_object = datetime.fromtimestamp(timestamp)
        if is_24h:
            return dt_object.strftime("%H:%M")
        else:
            return dt_object.strftime("%I:%M %p")
    


    # Fetch weather data using OpenWeather API
    def get_weather(self, city:str, pref:str='global')->dict[str, str]:
        # Distinguish user preference
        settings = {
            "global":{"metric":"metric", "temp_unit":"C", "dist_unit":"km", "spd_unit":"m/s", "is_24h":True},
            "us":{"metric":"imperial", "temp_unit":"F", "dist_unit":"mile", "spd_unit":"mph", "is_24h":False}
        }
        rule = settings[pref]

        # API call
        params = {"q":city, "appid":self.__weather_key, "units":rule["metric"]}
        json_data = self.make_api_request(self.weather_base_url, params)

        # Convert visibility
        visibility = int(json_data['visibility'] * (0.001 if pref == 'global' else 0.000621371))

        # Convert wind direction
        wind_deg = json_data['wind']['deg']
        wind_directions = ["North", "North-East", "East", "South-East", "South", "South-West", "West", "North-West"]
        wind_dir = wind_directions[round(wind_deg / 45) % 8]

        # Formatting data for presentation
        return {
            "city": city,
            "time": self.format_timestamp(json_data['dt'], rule['is_24h']),
            "temperature": f"{json_data['main']['temp']}°{rule['temp_unit']}",
            "min_temperature": f"{json_data['main']['temp_min']}°{rule['temp_unit']}",
            "max_temperature": f"{json_data['main']['temp_max']}°{rule['temp_unit']}",
            "feels": f"{json_data['main']['feels_like']}°{rule['temp_unit']}",
            "humidity": f"{json_data['main']['humidity']}%",
            "visibility": f"{visibility} {rule['dist_unit']}",
            "weather": json_data['weather'][0]['main'],
            "weather_icon": f"http://openweathermap.org/img/wn/{json_data['weather'][0]['icon']}.png",
            "wind_speed": f"{json_data['wind']['speed']} {rule['spd_unit']}",
            "wind_dir": wind_dir,
            "sunrise": self.format_timestamp(json_data['sys']['sunrise'], rule['is_24h']),
            "sunset": self.format_timestamp(json_data['sys']['sunset'], rule['is_24h'])
        }
    


    # Fetch flight prices from Amadeus API
    def get_flight_offers(
            self, 
            origin:str, 
            destination:str, 
            departure_date:str, 
            adults:int=1, 
            max_results:int=10
        )->pd.DataFrame:
        # Error check
        if not self.__amadeus_token:
            return {"error": "Failed to authenticate with Amadeus API"}

        # Parameter setting
        headers = {"Authorization":f"Bearer {self.__amadeus_token}"}
        params = {
            "originLocationCode":origin,
            "destinationLocationCode":destination,
            "departureDate":departure_date,
            "adults":adults,
            "max": max_results
        }

        # API call
        json_data = self.make_api_request(self.amadeus_flight_url, params, headers)

        # Extract important information
        flight_offers = []
        for flight in json_data['data']:
            # Airline and price information
            airline_code = flight["validatingAirlineCodes"][0]
            airline_name = json_data["dictionaries"]["carriers"][airline_code]
            price = float(flight["price"]['grandTotal'])
            price_unit = flight["price"]['currency']
            
            # Flight itinerary details
            itinerary = flight["itineraries"][0]
            segments = itinerary["segments"]
            # Extract departure and arrival times
            first_segment = segments[0]
            last_segment = segments[-1]
            departure_time = first_segment["departure"]["at"]
            arrival_time = last_segment["arrival"]["at"]
            # Check if the flight is direct or requires transits
            is_direct = len(segments) == 1
            transit_info = 'Direct flight' if is_direct else 'Need to transfer'

            # Append formatted flight information
            flight_offers.append({
                "flight_num": f'{first_segment["carrierCode"]}{first_segment["number"]}',
                "airline": airline_name,
                "price": price,  # Store as float for sorting
                "price_display": f"{price} {price_unit}",
                "departure_time": departure_time,
                "arrival_time": arrival_time,
                "transit_info": transit_info
            })

        # Sort flights by price (ascending order)
        flight_offers.sort(key=lambda x: x["price"])

        # Clean up
        for flight in flight_offers:
            del flight["price"]
        df_flight_info = pd.DataFrame(flight_offers)
        df_flight_info["airline"] = df_flight_info["airline"].str.upper()

        # Combine with Wiki scraped info
        df_airlines = pd.read_csv('airlines_data.csv')
        df_airlines.columns = ["Airline", "Country of Operation"]
        df_airlines["Airline"] = df_airlines["Airline"].str.replace(r"\[.*?\]", "", regex=True).str.strip()
        df_airlines["Airline"] = df_airlines["Airline"].str.upper()

        df_flight_info = df_flight_info.merge(df_airlines, left_on="airline", right_on="Airline", how="left")
        df_flight_info = df_flight_info.drop(columns=["Airline"])

        df_flight_info = df_flight_info[
            ["flight_num", "airline", "Country of Operation", 
             "price_display", "departure_time", "arrival_time", "transit_info"]
        ]
        return df_flight_info



    # Fetch hotel options from Booking.com
    def get_hotel_list(
            self, 
            city_name:str,
            star:list[int],
            adults:int,
            rooms:int,
            children:int,
            checkin:str,
            checkout:str,
            currency_code:str='USD',
            language:str='en-us',
            sort_logic:str='review_count')->pd.DataFrame:
        # Use web scraping technique to retreive hotel info
        fetch_result = get_hotel_detail(
            city_name, checkin, checkout, adults, children, rooms, star, currency_code, language, True
        )

        # Sort hotel info
        sorted_hotel_list = pd.DataFrame(fetch_result).sort_values(by=sort_logic, ascending=False)

        return sorted_hotel_list

<br/>

### Function Test - Setup

In [11]:
apis = ext_api('api_keys.txt')

stp = SmartTravelPlanner(apis['openweathermap_api'], (apis['amadeus_api'], apis['amadeus_secret']))

<br/>

### Function Test - Weather Information

In [5]:
city_weather = stp.get_weather("London", pref='us')
print(city_weather)

{'city': 'London', 'time': '03:06 PM', 'temperature': '37.63°F', 'min_temperature': '35.02°F', 'max_temperature': '39.61°F', 'feels': '33.12°F', 'humidity': '91%', 'visibility': '6 mile', 'weather': 'Clouds', 'weather_icon': 'http://openweathermap.org/img/wn/04n.png', 'wind_speed': '5.75 mph', 'wind_dir': 'East', 'sunrise': '11:27 PM', 'sunset': '09:02 AM'}


<br/>

### Function Test - Flight Information

In [13]:
flight_info = stp.get_flight_offers('SFO', 'LHR', '2025-03-28')
flight_info

,flight_num,airline,Country of Operation,price_display,departure_time,arrival_time,transit_info
0,VS4507,VIRGIN ATLANTIC,Guernsey,415.07 EUR,2025-03-28T13:35:00,2025-03-29T10:00:00,Need to transfer
1,VS4507,VIRGIN ATLANTIC,the United Kingdom,415.07 EUR,2025-03-28T13:35:00,2025-03-29T10:00:00,Need to transfer
2,VS1904,VIRGIN ATLANTIC,Guernsey,415.07 EUR,2025-03-28T16:45:00,2025-03-29T13:10:00,Need to transfer
3,VS1904,VIRGIN ATLANTIC,the United Kingdom,415.07 EUR,2025-03-28T16:45:00,2025-03-29T13:10:00,Need to transfer
4,VS5344,VIRGIN ATLANTIC,Guernsey,415.07 EUR,2025-03-28T14:10:00,2025-03-29T11:40:00,Need to transfer
5,VS5344,VIRGIN ATLANTIC,the United Kingdom,415.07 EUR,2025-03-28T14:10:00,2025-03-29T11:40:00,Need to transfer
6,VS5342,VIRGIN ATLANTIC,Guernsey,415.07 EUR,2025-03-28T11:55:00,2025-03-29T10:35:00,Need to transfer
7,VS5342,VIRGIN ATLANTIC,the United Kingdom,415.07 EUR,2025-03-28T11:55:00,2025-03-29T10:35:00,Need to transfer
8,VS5342,VIRGIN ATLANTIC,Guernsey,415.07 EUR,2025-03-28T11:55:00,2025-03-29T11:40:00,Need to transfer
9,VS5342,VIRGIN ATLANTIC,the United Kingdom,415.07 EUR,2025-03-28T11:55:00,2025-03-29T11:40:00,Need to transfer


<br/>

### Function Test - Hotel Information

In [9]:
hotel_list = stp.get_hotel_list('London', [4, 5], 2, 1, 0, '2025-03-28', '2025-03-30')
hotel_list

Sign-in modal did not appear, continuing...


,hotel_name,review_score,review_count,price
10,Park Plaza London Westminster Bridge,8.4,24737,$469
17,Leonardo Royal London St Paul’s,8.7,14739,$435
8,The Dilly,8.3,10766,$528
5,Holiday Inn London Kensington High St. by IHG,7.6,10724,$366
14,"Radisson Blu Hotel, London Canary Wharf East",8.5,9938,$380
2,citizenM Tower of London,8.6,8636,$349
11,One Hundred Shoreditch,8.7,6825,$419
15,"Montcalm Piccadilly Townhouse, London West End",8.5,5716,$427
20,"Radisson Blu Hotel, London Bloomsbury",7.7,5687,$554
19,Royal Lancaster London,9.1,5450,$636


<br/>

### Work & Design Plan

#### **Project Overview**
The **Smart Travel Planner with Real-time Data** is designed to assist travelers by providing real-time information on weather, flights, and hotel information. The project integrates multiple APIs to offer a comfortable user experience for trip planning.

#### **Goals and Objectives**
- Provide travelers with accurate and up-to-date information.
- Simplify the process of comparing flight and hotel options.
- Offer a comfortable experience with intuitive navigation and clear results.

#### **Scope of Work**
- Collect and display real-time weather updates.
- Aggregate and compare flight prices from various sources.
- Present hotel for informed decision-making.

#### **Key Functions**
- Retrieve real-time weather data for the destination.
- Fetch and compare flight prices from the user’s departure city.
- Provide customer-rated hotel options in the destination.

<br/>

### Challenges

#### **API Registration Issues**
Initially, we planned to use the **Skyscanner API** for retrieving flight prices and rental car options. However, the platform required commercial usage of their API only.
- **Solved:** Turn to **Amadeus API** instead.

#### **Difficulties in Fetching Hotel Data Efficiently**
When retrieving hotel data from the **Amadeus API**, we could not fetch detailed hotel info in a single request. The API seemed to have interanl hotel code conflict that made us failed in combining hotel basic info and detailed info.
- **Solved:** Apply web scraping technique on *<u>Booking.com</u>* to fetch all required info at once.
